In [5]:
import pandas as pd
import numpy as np
import ast
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

print("--- 1. Đang xử lý dữ liệu ---")
df = pd.read_csv('data_train.csv')
df['tags_list'] = df['tags_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

all_tags_flat = [tag for sublist in df['tags_list'] for tag in sublist]
top_20_tags = [tag for tag, count in Counter(all_tags_flat).most_common(20)]
top_20_set = set(top_20_tags)
print(f"Top 20 Tags: {top_20_tags}")

print("\n--- 2. Xây dựng Ma trận tương đồng (Soft Metric) ---")
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(df['text_content'])

def get_tag_centroids(df, X_tfidf, all_unique_tags):
    tag_centroids = {}
    for tag in all_unique_tags:
        indices = df[df['tags_list'].apply(lambda tags: tag in tags)].index
        if len(indices) > 0:
            centroid = np.mean(X_tfidf[indices], axis=0)
            tag_centroids[tag] = np.asarray(centroid).flatten()
    return tag_centroids

all_unique_tags = list(set(all_tags_flat))
tag_centroids = get_tag_centroids(df, X_tfidf, all_unique_tags)

def calculate_tag_similarity(tag_centroids, top_20, all_tags):
    similarity_map = {}
    valid_tags = [t for t in all_tags if t in tag_centroids]
    valid_top_20 = [t for t in top_20 if t in tag_centroids]
    if not valid_tags or not valid_top_20: return {}
    vectors_all = np.array([tag_centroids[t] for t in valid_tags])
    vectors_top20 = np.array([tag_centroids[t] for t in valid_top_20])
    sim_matrix = cosine_similarity(vectors_all, vectors_top20)
    for i, tag_rare in enumerate(valid_tags):
        similarity_map[tag_rare] = {}
        for j, tag_top in enumerate(valid_top_20):
            similarity_map[tag_rare][tag_top] = sim_matrix[i][j]
    return similarity_map

sim_map = calculate_tag_similarity(tag_centroids, top_20_tags, all_unique_tags)

if 'ai' in top_20_tags:
    print("\nVí dụ: Top 3 tag giống 'ai' nhất:")
    ai_sims = []
    for t in all_unique_tags:
        if t in sim_map and 'ai' in sim_map[t]:
            if t != 'ai':
                ai_sims.append((t, sim_map[t]['ai']))
    print(sorted(ai_sims, key=lambda x: x[1], reverse=True)[:3])

df['filtered_tags'] = df['tags_list'].apply(lambda tags: [t for t in tags if t in top_20_set])
df_train_clean = df[df['filtered_tags'].map(len) > 0].reset_index(drop=True)

X = df_train_clean['text_content']
y = df_train_clean['filtered_tags']

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, df_train_clean.index, test_size=0.2, random_state=42
)

y_test_original_tags = df.iloc[df_train_clean.iloc[idx_test].index]['tags_list'].values

tfidf = TfidfVectorizer(max_features=5000)
svd = TruncatedSVD(n_components=200)
X_train_vec = svd.fit_transform(tfidf.fit_transform(X_train))
X_test_vec = svd.transform(tfidf.transform(X_test))

mlb = MultiLabelBinarizer(classes=top_20_tags)
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)

print("\n--- 3. Training XGBoost ---")
clf = OneVsRestClassifier(XGBClassifier(n_estimators=100, tree_method='hist', n_jobs=-1))
clf.fit(X_train_vec, y_train_bin)

y_train_pred_bin = clf.predict(X_train_vec)
y_pred_bin = clf.predict(X_test_vec)
y_train_pred_tags = mlb.inverse_transform(y_train_pred_bin)
y_pred_tags = mlb.inverse_transform(y_pred_bin)

print("\n--- 4. Kết quả ---")
def calculate_soft_score(pred_tags_list, original_tags_list, sim_map, threshold=0.7):
    soft_precisions = []
    for preds, originals in zip(pred_tags_list, original_tags_list):
        if len(preds) == 0:
            soft_precisions.append(0)
            continue
        hits = 0
        for p_tag in preds:
            is_match = False
            if p_tag in originals:
                is_match = True
            else:
                for o_tag in originals:
                    if o_tag in sim_map and p_tag in sim_map[o_tag]:
                        if sim_map[o_tag][p_tag] >= threshold:
                            is_match = True
                            break
            if is_match: hits += 1
        soft_precisions.append(hits / len(preds))
    return np.mean(soft_precisions)

y_test_top20 = y_test.values
hard_correct_train = sum([set(p) == set(t) for p, t in zip(y_train_pred_tags, y_train.values)])
hard_correct_test = sum([set(p) == set(t) for p, t in zip(y_pred_tags, y_test_top20)])
print(f"Hard Accuracy Train: {hard_correct_train/len(y_train):.4f}")
print(f"Hard Accuracy Test: {hard_correct_test/len(y_test):.4f}")

soft_score_train = calculate_soft_score(y_train_pred_tags, y_train.values, sim_map, threshold=0.7)
soft_score_test = calculate_soft_score(y_pred_tags, y_test_original_tags, sim_map, threshold=0.7)
print(f"Soft Precision Train (threshold=0.7): {soft_score_train:.4f}")
print(f"Soft Precision Test (threshold=0.7): {soft_score_test:.4f}")


--- 1. Đang xử lý dữ liệu ---
Top 20 Tags: ['ai', 'javascript', 'development', 'aws', 'devops', 'reconnection', 'react', 'java', 'go', 'kubernetes', 'nodejs', 'cometapi', 'docker', 'android', 'database', 'python', 'security', 'blockchain', 'php', 'typescript']

--- 2. Xây dựng Ma trận tương đồng (Soft Metric) ---

Ví dụ: Top 3 tag giống 'ai' nhất:
[('reconnection', 0.6486177210112248), ('development', 0.5854952003921959), ('python', 0.532715829245051)]

--- 3. Training XGBoost ---

--- 4. Kết quả ---
Hard Accuracy Train: 1.0000
Hard Accuracy Test: 0.5008
Soft Precision Train (threshold=0.7): 1.0000
Soft Precision Test (threshold=0.7): 0.6206


In [9]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# =============================================================================
# 1. LOAD DỮ LIỆU
# =============================================================================
print("--- 1. Đang xử lý dữ liệu ---")
df = pd.read_csv('data_train.csv')
# Parse chuỗi list thành list thật
df['tags_list'] = df['tags_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Bỏ dòng rỗng
df = df[df['tags_list'].map(len) > 0].reset_index(drop=True)

# Chia tập Train/Test
# Lưu ý: Ta dùng toàn bộ tag, KHÔNG lọc Top 20 nữa để tận dụng tối đa dữ liệu
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['text_content'], df['tags_list'], test_size=0.2, random_state=42
)

# =============================================================================
# 2. VECTOR HÓA (TF-IDF + SVD)
# =============================================================================
print("\n--- 2. Tạo không gian Vector ngữ nghĩa ---")
# TF-IDF
tfidf = TfidfVectorizer(max_features=10000, stop_words='english') 
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

# SVD (Giảm chiều để tính khoảng cách chuẩn hơn)
# SVD giúp vector 'xe hơi' và 'ô tô' xích lại gần nhau hơn
svd = TruncatedSVD(n_components=200, random_state=42)
X_train_vec = svd.fit_transform(X_train_tfidf)
X_test_vec = svd.transform(X_test_tfidf)

print(f"Không gian vector: {X_train_vec.shape}")

# =============================================================================
# 3. HỌC VỊ TRÍ CỦA CÁC TAG (TAG CENTROIDS)
# =============================================================================
print("\n--- 3. Tính toán vị trí trung tâm (Centroids) của từng Tag ---")
tag_centroids = {}
tag_counts = {}

# Duyệt qua từng bài train để cộng dồn vector cho tag tương ứng
# X_train_vec là numpy array, y_train là Series
y_train_values = y_train.values

for i in range(len(y_train_values)):
    vec = X_train_vec[i]
    tags = y_train_values[i]
    for tag in tags:
        if tag not in tag_centroids:
            tag_centroids[tag] = np.zeros(200) # 200 là n_components của SVD
            tag_counts[tag] = 0
        tag_centroids[tag] += vec
        tag_counts[tag] += 1

# Chia trung bình để ra Centroid
unique_tags = []
centroid_vectors = []

for tag in tag_centroids:
    tag_centroids[tag] /= tag_counts[tag]
    unique_tags.append(tag)
    centroid_vectors.append(tag_centroids[tag])

centroid_matrix = np.array(centroid_vectors)
print(f"Đã học được vị trí của {len(unique_tags)} tags.")

# =============================================================================
# 4. PHÂN CỤM TAG (CLUSTERING) - ĐỂ LÀM SOFT METRIC
# =============================================================================
print("\n--- 4. Phân cụm các Tags (Tạo nhóm ngữ nghĩa) ---")
# Số lượng cụm = 1/5 tổng số tag (hoặc tối đa 50)
n_clusters = min(50, len(unique_tags) // 2)
if n_clusters < 2: n_clusters = 2

kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
tag_clusters = kmeans.fit_predict(centroid_matrix)

# Map: Tag -> Cluster ID
tag_to_cluster = {tag: cid for tag, cid in zip(unique_tags, tag_clusters)}

# =============================================================================
# 5. DỰ ĐOÁN & ĐÁNH GIÁ (Dựa trên khoảng cách)
# =============================================================================
print("\n--- 5. Dự đoán trên tập Test ---")

# Tính độ tương đồng (Cosine Similarity) giữa Bài viết Test và Tất cả Tags
# Kết quả là ma trận (Số bài test x Số lượng tags)
similarity_matrix = cosine_similarity(X_test_vec, centroid_matrix)

# Lấy Top 3 Tags gần nhất cho mỗi bài
top_k = 3
# argsort trả về index của tag, lấy từ thấp đến cao nên cần đảo ngược [::-1]
top_k_indices = np.argsort(similarity_matrix, axis=1)[:, ::-1][:, :top_k]

y_pred = []
for idx_row in top_k_indices:
    # Map từ index quay lại tên Tag
    pred_tags = [unique_tags[idx] for idx in idx_row]
    y_pred.append(pred_tags)

# --- ĐÁNH GIÁ ---
print("\n--- KẾT QUẢ ĐÁNH GIÁ ---")

# Hàm tính độ chính xác dựa trên Cluster (Soft Metric)
def evaluate_soft_cluster(y_true, y_pred, tag_to_cluster):
    correct_count = 0
    total = 0
    
    for true_tags, pred_tags in zip(y_true, y_pred):
        if not pred_tags: continue
        
        # 1. Lấy tất cả Cluster ID của nhãn thật
        true_cluster_ids = set()
        for t in true_tags:
            if t in tag_to_cluster:
                true_cluster_ids.add(tag_to_cluster[t])
        
        # 2. Kiểm tra xem Tag dự đoán CAO NHẤT (Top 1) có thuộc cụm đúng không?
        top_1_pred = pred_tags[0]
        pred_cluster = tag_to_cluster.get(top_1_pred, -1)
        
        if pred_cluster in true_cluster_ids:
            correct_count += 1
        
        total += 1
        
    return correct_count / total

soft_acc = evaluate_soft_cluster(y_test.values, y_pred, tag_to_cluster)

print(f"Tổng số bài test: {len(y_test)}")
print(f"Cluster-based Soft Accuracy: {soft_acc:.4f}")
print("(Nghĩa là: {:.2f}% số bài viết được gán nhãn ĐÚNG CHỦ ĐỀ/NHÓM NGỮ NGHĨA)".format(soft_acc * 100))

# In ví dụ minh họa
print("\n--- Ví dụ thực tế ---")
for i in range(5):
    true_t = y_test.values[i]
    pred_t = y_pred[i] # Top 3 dự đoán
    
    # Lấy Cluster ID để so sánh
    true_c = [tag_to_cluster.get(t, 'N/A') for t in true_t]
    pred_c_top1 = tag_to_cluster.get(pred_t[0], 'N/A')
    
    match_status = "ĐÚNG CỤM" if pred_c_top1 in true_c else "SAI"
    
    print(f"Bài {i}:")
    print(f"  - Tag thật: {true_t} (Cụm ID: {true_c})")
    print(f"  - Dự đoán:  {pred_t} (Top 1 Cụm ID: {pred_c_top1})")
    print(f"  => Kết quả: {match_status}")
    print("-" * 30)

--- 1. Đang xử lý dữ liệu ---

--- 2. Tạo không gian Vector ngữ nghĩa ---
Không gian vector: (2619, 200)

--- 3. Tính toán vị trí trung tâm (Centroids) của từng Tag ---
Đã học được vị trí của 20 tags.

--- 4. Phân cụm các Tags (Tạo nhóm ngữ nghĩa) ---

--- 5. Dự đoán trên tập Test ---

--- KẾT QUẢ ĐÁNH GIÁ ---
Tổng số bài test: 655
Cluster-based Soft Accuracy: 0.8595
(Nghĩa là: 85.95% số bài viết được gán nhãn ĐÚNG CHỦ ĐỀ/NHÓM NGỮ NGHĨA)

--- Ví dụ thực tế ---
Bài 0:
  - Tag thật: ['development', 'javascript'] (Cụm ID: [1, 1])
  - Dự đoán:  ['java', 'development', 'reconnection'] (Top 1 Cụm ID: 1)
  => Kết quả: ĐÚNG CỤM
------------------------------
Bài 1:
  - Tag thật: ['ai'] (Cụm ID: [1])
  - Dự đoán:  ['ai', 'reconnection', 'development'] (Top 1 Cụm ID: 1)
  => Kết quả: ĐÚNG CỤM
------------------------------
Bài 2:
  - Tag thật: ['security'] (Cụm ID: [1])
  - Dự đoán:  ['security', 'nodejs', 'javascript'] (Top 1 Cụm ID: 1)
  => Kết quả: ĐÚNG CỤM
------------------------------
Bài 